In [ ]:
from ultralytics import YOLO
model=YOLO("license_plate_detector.pt")
# results=model.predict("test_image",conf=0.25,save=True)



image 1/1 c:\Users\bhavy\OneDrive\Desktop\Deep_learning\DL_projects\Yolo_prjct\Numberplate\test_image\vehicle-identification-plate.webp: 448x640 1 license_plate, 624.4ms
Speed: 4.9ms preprocess, 624.4ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)
Results saved to C:\Users\bhavy\OneDrive\Desktop\Deep_learning\DL_projects\Yolo_prjct\Numberplate\runs\detect\predict


In [ ]:
import cv2
import pytesseract
from ultralytics import YOLO

# 🔹 Tesseract path set cheyyu (important)
pytesseract.pytesseract.tesseract_cmd = r"D:\Luminar\Study\Deep learning\DL Project\OCR\Tesseract-OCR\tesseract.exe"
# 🔹 Load YOLO model
model = YOLO("best.pt")

# 🔹 Image path
image_path = r"test_image\vehicle-identification-plate.webp"

# 🔹 Prediction
results = model.predict(source=image_path, conf=0.25)

# 🔹 Original image read
img = cv2.imread(image_path)

boxes = results[0].boxes

if boxes is not None and len(boxes) > 0:
    # First detected plate
    x1, y1, x2, y2 = boxes.xyxy[0].cpu().numpy().astype(int)

    # Crop plate
    plate = img[y1:y2, x1:x2]

    # Save cropped plate
    cv2.imwrite("number_plate_crop.jpg", plate)

    # 🔹 Preprocess for better OCR
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

    # 🔹 OCR
    text = pytesseract.image_to_string(
        thresh,
        config="--psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    )

    print("✅ Detected Number Plate Text:", text.strip())

else:
    print("❌ Number plate not detected")


image 1/1 c:\Users\bhavy\OneDrive\Desktop\Deep_learning\DL_projects\Yolo_prjct\Numberplate\test_image\vehicle-identification-plate.webp: 448x640 1 license_plate, 621.2ms
Speed: 2.9ms preprocess, 621.2ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)
✅ Detected Number Plate Text: TO7U5U50


# video


In [5]:
import cv2
import pytesseract
from ultralytics import YOLO

# 🔹 Tesseract path
pytesseract.pytesseract.tesseract_cmd = r"D:\Luminar\Study\Deep learning\DL Project\OCR\Tesseract-OCR\tesseract.exe"

# 🔹 Load YOLO model
model = YOLO("license_plate_detector.pt")

# 🔹 Video path
video_path = r"Traffic Stock Video (1).mov"


In [7]:

cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 440))  # width, height
    frame =cv2.flip(frame,1)


    detected_texts = []  # store multiple plates

    # 🔹 YOLO detection
    results = model.predict(source=frame, conf=0.25, verbose=False)
    boxes = results[0].boxes

    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

            plate = frame[y1:y2, x1:x2]
            if plate.size == 0:
                continue

            # 🔹 OCR preprocessing
            gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
            gray = cv2.bilateralFilter(gray, 11, 17, 17)
            _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

            text = pytesseract.image_to_string(
                thresh,
                config="--psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
            ).strip()

            if text:
                detected_texts.append(text)

            # Optional: draw box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)

    # 🔹 Display detected plates at TOP-LEFT
    y_offset = 30
    for i, txt in enumerate(detected_texts):
        cv2.putText(
            frame,
            f"{i+1}. {txt}",
            (10, y_offset),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 0, 255),
            2
        )
        y_offset += 30

    cv2.imshow("Number Plate Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



In [8]:
cap.release()
cv2.destroyAllWindows()

confidance score 

In [17]:
video_path = r"Traffic Stock Video (2).mov"

In [18]:
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (640, 440))
    frame = cv2.flip(frame, 1)

    # 🔹 YOLO detection
    results = model.predict(source=frame, conf=0.25, verbose=False)
    boxes = results[0].boxes

    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])  # confidence score

            label = f"Plate: {conf * 100:.1f}%"

            # 🔹 Draw thin bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 1)

            # 🔹 Draw confidence text
            cv2.putText(
                frame,
                label,
                (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 0, 0),
                1,
                cv2.LINE_AA
            )

    cv2.imshow("Number Plate Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
